<center><b>Deep Learning - Convolutional Network Ensemble (Average)</b></center><br>

The code below implements multiple convolutional networks and utilises them in an ensemble to improve prediction. The ensemble method used here is simply averaging with a slight adjustment based on the accuracy of the architecture. The predicted weights from each architecture are multiplied so stronger performing models' prediction get a higher weight then they are added together and the class with the highest prediction is selected.<br> <br>
The dataset is very small, it contains 17 categories but only 1020 images in total. To adjust for this, data augmentation is implemented also the data is only split into 'training' and 'validation' sets, 'test' sets are not being utilised.<br> <br>
Data checkpointing is also used that caches the best model with the smallest validation loss, this allows incremental training. Custom data checkpointing was used to compensate for a shortcoming of Keras' built-in data checkpointer as that system loses the information about the validation loss when loading the data back in. This custom checkpointer saves the validation loss into a separate file.<br>
<br>
As can be seen on the results, the ensemble method resulted in a better performance than any of the architectures did by themselves.

In [1]:
import os.path
import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
import matplotlib.pyplot as plt
import zipfile

import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.


2.1.0


In [2]:
# Handle dataset
FILE_PATH = os.getcwd()
CACHE_PATH = FILE_PATH+"/cached/deeplearning/"
data_file = FILE_PATH+"/data/DeepLearning/conv_net_data.zip"

data = zipfile.ZipFile(data_file)
data_file = data.open('data1.h5')

def loadDataH5():
    with h5py.File(data_file,'r') as hf:
      trainX = np.array(hf.get('trainX'))
      trainY = np.array(hf.get('trainY'))
      valX = np.array(hf.get('valX'))
      valY = np.array(hf.get('valY'))
      print (trainX.shape,trainY.shape)
      print (valX.shape,valY.shape)
      return trainX, trainY, valX, valY 
 
trainX, trainY, testX, testY = loadDataH5() 

(1020, 128, 128, 3) (1020,)
(340, 128, 128, 3) (340,)


In [3]:
# const variables about the dataset
IMG_DEPTH   = 3
IMG_WIDTH   = 128
IMG_HEIGHT  = 128
NUM_CLASSES = 17


NUM_EPOCHS  = 50

In [4]:
# All Architectures used

def create_case_0_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    # add conv layer
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_case_1_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_case_2_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 3
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 3
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_case_3_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 3
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 3
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 4
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    # add pooling layer 4
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(512, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_alex_net_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add conv layer 1
    model.add(tf.keras.layers.Conv2D(96, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    # add pooling layer 1
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 2
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    # add pooling layer 2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    # add conv layer 3
    model.add(tf.keras.layers.Conv2D(384, (3, 3), padding="same", activation='relu'))
    # add conv layer 4
    model.add(tf.keras.layers.Conv2D(384, (3, 3), padding="same", activation='relu'))
    # add conv layer 5
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    # add pooling layer 3
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dense(1024, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

def create_vgg_16_architecture(width, height, depth, classes):
    inputShape = (height, width, depth)

    model = tf.keras.Sequential()
    
    # add 2 conv layers
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))
    
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    
    # add 2 conv layers
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", activation='relu'))
    
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    
    # add 3 conv layers
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    
    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add 3 conv layers
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))

    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add 3 conv layers
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), padding="same", activation='relu'))

    # add pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

    # add fully connected layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(4096, activation='relu'))
    model.add(keras.layers.Dense(4096, activation='relu'))
    # add softmax layer
    model.add(keras.layers.Dense(classes, activation='softmax'))
    return model

In [5]:
# Base class for all architectures to derive from. Handles checkpointing and data augmentation
class CNN_base_augmented_checkpointed:
  def __init__(self, try_to_load_weights=True, ext_name=""):
    # Initialise Variables
    self.BATCH_SIZE = 16
  
    # Initialise model
    self.model = self.create_model(IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH, NUM_CLASSES)
    self.model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])

    # Saved data file names
    self.weights_file_name = "{}/{}{}.hdf5".format(CACHE_PATH, self.get_name(), ext_name)
    self.val_loss_file_name = "{}/{}{}_val_loss.txt".format(CACHE_PATH, self.get_name(), ext_name)

    # The current best val_loss
    self.best_val_loss = float("inf")
    if try_to_load_weights:
      self.load_checkpointed_weights()
    

  def load_best_val_loss_value(self):
    """
    This function is used to load the validation loss of the best model.
    It is stored in a txt file when the best model's weights were cached
    """
    if os.path.isfile(self.val_loss_file_name):
      loss_file = open(self.val_loss_file_name, "r")
      self.best_val_loss = float(loss_file.read())
      loss_file.close()
      print("{} - Loaded best val_loss value: {:.4f} from: {}".format(self.get_name(), self.best_val_loss, self.val_loss_file_name))


  def save_weights_callback(self, logs):
    """
    This is a custom callback function to be used to cache the weights of the network when val_loss decreases
    """
    val_loss = logs.get('val_loss')
    if val_loss < self.best_val_loss:
      print("\n{} - Caching Checkpoint - val_loss has improved from: {:.4f} to: {:.4f}".format(self.get_name(), self.best_val_loss, val_loss))
      self.best_val_loss = val_loss
      
      self.model.save_weights(self.weights_file_name)

      loss_file = open(self.val_loss_file_name,"w")
      loss_file.write(str(val_loss))
      loss_file.close()
    else:
      print("\n{} Val_loss did not improve. Best is: {:.4f}. Not Caching".format(self.get_name(), self.best_val_loss))

  def train(self, tr_x, tr_y, val_x, val_y, num_epochs=NUM_EPOCHS):
    print("{} - Training for {} epochs".format(self.get_name(), num_epochs))

    # Create the image generator for Data Augmentation
    datagen_batch = len(tr_x) / self.BATCH_SIZE
    datagen = self.create_image_generator()

    # Create the model checkpointer
    custom_checkpointing_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs:self.save_weights_callback(logs))

    # Train
    self.model.fit(datagen.flow(tr_x, tr_y, batch_size=self.BATCH_SIZE), steps_per_epoch=datagen_batch, epochs=num_epochs, validation_data=(val_x, val_y), callbacks=[custom_checkpointing_callback])

  def load_checkpointed_weights(self):
    # If possible load the previous training results. Weights and assosiated val_loss
    if os.path.isfile(self.weights_file_name):
      print("{} - Loading previous training data".format(self.get_name()))
      self.model.load_weights(self.weights_file_name)
      print("{} - Loaded previous model weights from: {}".format(self.get_name(), self.weights_file_name))
      self.load_best_val_loss_value()

  def get_accuracy_multiplier(self, tr_x, tr_y):
    # Get Prediction
    pred_y = self.predict(tr_x)
    # Calculate Accuracy
    pred_y = tf.math.argmax(pred_y, 1)
    match_array = tf.equal(pred_y, tr_y)
    accuracy = tf.reduce_sum(tf.cast(match_array, tf.float32)) / len(tr_y)
    return 1.0 + accuracy

  def predict(self, val_x):
    return self.model.predict(val_x)

  def create_model(self, width, height, depth, classes):
    raise NotImplementedError()

  def create_image_generator(self):
    raise NotImplementedError()

  def get_name(self):
    raise NotImplementedError()

In [6]:
# Similar to Alex-net
class alex_net_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Alex_Net"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=40,
       width_shift_range=0.2,
       height_shift_range=0.2,
       horizontal_flip=True,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_alex_net_architecture(width, height, depth, classes)


In [7]:
# Similar to VGG 16
class vgg_16_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "VGG_16"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=30,
       width_shift_range=0.3,
       height_shift_range=0.3,
       horizontal_flip=False,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_vgg_16_architecture(width, height, depth, classes)

In [8]:
# Case 0
class case_0_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_0"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=20,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_case_0_architecture(width, height, depth, classes)

In [9]:
# Case 1 
class case_1_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_1"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=50,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True,
       vertical_flip=False)


  def create_model(self, width, height, depth, classes):
    return create_case_1_architecture(width, height, depth, classes)

In [10]:
# Case 2
class case_2_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_2"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=75,
       width_shift_range=0.4,
       height_shift_range=0.4,
       horizontal_flip=True,
       vertical_flip=False,
       zoom_range=0.1)


  def create_model(self, width, height, depth, classes):
    return create_case_2_architecture(width, height, depth, classes)

In [11]:
# Case 3
class case_3_CNN_augmented_checkpointed(CNN_base_augmented_checkpointed):
  def get_name(self):
    return "Case_3"

  def create_image_generator(self):
   return tf.keras.preprocessing.image.ImageDataGenerator(
       rotation_range=10,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True,
       vertical_flip=False,
       zoom_range=0.3)


  def create_model(self, width, height, depth, classes):
    return create_case_3_architecture(width, height, depth, classes)

In [12]:
# Helper function to calculate loss and accuracy
def get_prediction_result(predicted_results, y_labels):
  # Calculate Loss
  y_labels_hot = np_utils.to_categorical(y_labels, NUM_CLASSES)
  predicted_results_clipped = tf.clip_by_value(predicted_results, 1e-10, 1.0)
  loss = (1.0/predicted_results_clipped.shape[0]) * tf.math.reduce_sum(-tf.math.multiply(tf.math.log(predicted_results_clipped),y_labels_hot))

  # Calculate Accuracy
  predicted_y = tf.math.argmax(predicted_results, 1)
  match_array = tf.equal(predicted_y, y_labels)
  accuracy = tf.reduce_sum(tf.cast(match_array, tf.float32)) / len(y_labels)

  return (loss, accuracy)

In [14]:
# Create Ensemble method
# This setup allows incremental training

SHOULD_TRAIN = True         # true if models should be trained
TRAIN_FOR_NUM_EPOCHS = 15   # the number of epochs to run if training
LOAD_WEIGHTS = True         # set to false to restart training



# 1. Create the used architectures. This will also load the cached best model if exists
print("Initialising CNN architectures")
alex_net   = alex_net_CNN_augmented_checkpointed(LOAD_WEIGHTS)
vgg_16_net = vgg_16_CNN_augmented_checkpointed(LOAD_WEIGHTS)
case_0_net = case_0_CNN_augmented_checkpointed(LOAD_WEIGHTS)
case_1_net = case_1_CNN_augmented_checkpointed(LOAD_WEIGHTS)
case_2_net = case_2_CNN_augmented_checkpointed(LOAD_WEIGHTS)
case_3_net = case_3_CNN_augmented_checkpointed(LOAD_WEIGHTS)


if SHOULD_TRAIN:
  # each model will be trained here. Once training is complete, the checkpointed weights
  # are loaded back in. This is to allow the final tests to be done on the best performing model
  print("Training CNN architectures for {} epochs".format(TRAIN_FOR_NUM_EPOCHS))
  alex_net.train(trainX, trainY, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  alex_net.load_checkpointed_weights()
  vgg_16_net.train(trainX, trainY, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  vgg_16_net.load_checkpointed_weights()
  case_0_net.train(trainX, trainY, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  case_0_net.load_checkpointed_weights()
  case_1_net.train(trainX, trainY, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  case_1_net.load_checkpointed_weights()
  case_2_net.train(trainX, trainY, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  case_2_net.load_checkpointed_weights()
  case_3_net.train(trainX, trainY, testX, testY, TRAIN_FOR_NUM_EPOCHS)
  case_3_net.load_checkpointed_weights()
  print("Training CNN architectures complete")

print("Gathering final results")

prediction_a = alex_net.predict(testX)
mult_a = alex_net.get_accuracy_multiplier(trainX, trainY)

prediction_v = vgg_16_net.predict(testX)
mult_v = vgg_16_net.get_accuracy_multiplier(trainX, trainY)

prediction_1 = case_1_net.predict(testX)
mult_1 = case_1_net.get_accuracy_multiplier(trainX, trainY)

prediction_2 = case_2_net.predict(testX)
mult_2 = case_2_net.get_accuracy_multiplier(trainX, trainY)

prediction_3 = case_3_net.predict(testX)
mult_3 = case_3_net.get_accuracy_multiplier(trainX, trainY)

prediction_0 = case_0_net.predict(testX)
mult_0 = case_0_net.get_accuracy_multiplier(trainX, trainY)

# Normalize the multiplier
sum_mult = mult_a + mult_v + mult_0 + mult_1 + mult_2 + mult_3
mult_a /= sum_mult
mult_v /= sum_mult
mult_0 /= sum_mult
mult_1 /= sum_mult
mult_2 /= sum_mult
mult_3 /= sum_mult

total_predictions = (prediction_a * mult_a) + (prediction_v * mult_v) + (prediction_0 * mult_0) + (prediction_1 * mult_1) + (prediction_2 * mult_2) + (prediction_3 * mult_3)

final_loss, final_accuracy = get_prediction_result(total_predictions, testY)

print("Predicting completed. Final Ensemble Loss: {:.4f} Final Ensemble Accuracy: {:.2f}%".format(final_loss, final_accuracy*100.0))

# Print Individual algorithm accuracy
alex_net_loss, alex_net_accuracy          = get_prediction_result(prediction_a, testY)
vgg_16_net_loss, vgg_16_net_accuracy      = get_prediction_result(prediction_v, testY)
case_0_net_loss, case_0_net_accuracy      = get_prediction_result(prediction_0, testY)
case_1_net_loss, case_1_net_accuracy      = get_prediction_result(prediction_1, testY)
case_2_net_loss, case_2_net_accuracy      = get_prediction_result(prediction_2, testY)
case_3_net_loss, case_3_net_accuracy      = get_prediction_result(prediction_3, testY)

print("Individual models loss and accuracy:")
print("Alex_net Loss: {:.4f} Accuracy: {:.2f}%".format(alex_net_loss, alex_net_accuracy*100.0))
print("VGG_16_net Loss: {:.4f} Accuracy: {:.2f}%".format(vgg_16_net_loss, vgg_16_net_accuracy*100.0))
print("Case_0_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_0_net_loss, case_0_net_accuracy*100.0))
print("Case_1_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_1_net_loss, case_1_net_accuracy*100.0))
print("Case_2_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_2_net_loss, case_2_net_accuracy*100.0))
print("Case_3_net Loss: {:.4f} Accuracy: {:.2f}%".format(case_3_net_loss, case_3_net_accuracy*100.0))


Initialising CNN architectures
Alex_Net - Loading previous training data
Alex_Net - Loaded previous model weights from: C:\Users\danie\Documents\Work\AI_portfolio\AI_portfolio/cached/deeplearning//Alex_Net.hdf5
Alex_Net - Loaded best val_loss value: 1.3948 from: C:\Users\danie\Documents\Work\AI_portfolio\AI_portfolio/cached/deeplearning//Alex_Net_val_loss.txt
VGG_16 - Loading previous training data
VGG_16 - Loaded previous model weights from: C:\Users\danie\Documents\Work\AI_portfolio\AI_portfolio/cached/deeplearning//VGG_16.hdf5
VGG_16 - Loaded best val_loss value: 1.4996 from: C:\Users\danie\Documents\Work\AI_portfolio\AI_portfolio/cached/deeplearning//VGG_16_val_loss.txt
Case_0 - Loading previous training data
Case_0 - Loaded previous model weights from: C:\Users\danie\Documents\Work\AI_portfolio\AI_portfolio/cached/deeplearning//Case_0.hdf5
Case_0 - Loaded best val_loss value: 1.0161 from: C:\Users\danie\Documents\Work\AI_portfolio\AI_portfolio/cached/deeplearning//Case_0_val_loss.

Epoch 5/15
63/63 [============================>.] - ETA: 3s - loss: 1.5785 - accuracy: 0.4343
VGG_16 Val_loss did not improve. Best is: 1.4897. Not Caching
64/63 [==============================] - 344s 5s/step - loss: 1.5747 - accuracy: 0.4353 - val_loss: 1.5619 - val_accuracy: 0.4206
Epoch 6/15
63/63 [============================>.] - ETA: 3s - loss: 1.5529 - accuracy: 0.4512
VGG_16 Val_loss did not improve. Best is: 1.4897. Not Caching
64/63 [==============================] - 356s 6s/step - loss: 1.5525 - accuracy: 0.4520 - val_loss: 1.6258 - val_accuracy: 0.4235
Epoch 7/15
63/63 [============================>.] - ETA: 3s - loss: 1.5780 - accuracy: 0.4283
VGG_16 Val_loss did not improve. Best is: 1.4897. Not Caching
64/63 [==============================] - 363s 6s/step - loss: 1.5767 - accuracy: 0.4304 - val_loss: 1.5204 - val_accuracy: 0.4412
Epoch 8/15
63/63 [============================>.] - ETA: 3s - loss: 1.5045 - accuracy: 0.4691
VGG_16 Val_loss did not improve. Best is: 1.4897

  ...
    to  
  ['...']
Train for 63.75 steps, validate on 340 samples
Epoch 1/15
63/63 [============================>.] - ETA: 0s - loss: 0.9595 - accuracy: 0.6793
Case_1 - Caching Checkpoint - val_loss has improved from: 1.0186 to: 1.0175
64/63 [==============================] - 48s 749ms/step - loss: 0.9563 - accuracy: 0.6804 - val_loss: 1.0175 - val_accuracy: 0.6618
Epoch 2/15
63/63 [============================>.] - ETA: 0s - loss: 0.9452 - accuracy: 0.6912
Case_1 Val_loss did not improve. Best is: 1.0175. Not Caching
64/63 [==============================] - 43s 667ms/step - loss: 0.9486 - accuracy: 0.6902 - val_loss: 1.0206 - val_accuracy: 0.6618
Epoch 3/15
63/63 [============================>.] - ETA: 0s - loss: 0.9562 - accuracy: 0.6863
Case_1 - Caching Checkpoint - val_loss has improved from: 1.0175 to: 1.0083
64/63 [==============================] - 39s 616ms/step - loss: 0.9552 - accuracy: 0.6863 - val_loss: 1.0083 - val_accuracy: 0.6647
Epoch 4/15
63/63 [==================

Epoch 12/15
63/63 [============================>.] - ETA: 0s - loss: 1.4115 - accuracy: 0.5269
Case_2 Val_loss did not improve. Best is: 1.2529. Not Caching
64/63 [==============================] - 36s 561ms/step - loss: 1.4085 - accuracy: 0.5265 - val_loss: 1.2763 - val_accuracy: 0.5941
Epoch 13/15
63/63 [============================>.] - ETA: 0s - loss: 1.3837 - accuracy: 0.5458
Case_2 Val_loss did not improve. Best is: 1.2529. Not Caching
64/63 [==============================] - 36s 560ms/step - loss: 1.3869 - accuracy: 0.5422 - val_loss: 1.2624 - val_accuracy: 0.5912
Epoch 14/15
63/63 [============================>.] - ETA: 0s - loss: 1.3802 - accuracy: 0.5488
Case_2 - Caching Checkpoint - val_loss has improved from: 1.2529 to: 1.2525
64/63 [==============================] - 38s 591ms/step - loss: 1.3819 - accuracy: 0.5461 - val_loss: 1.2525 - val_accuracy: 0.6118
Epoch 15/15
63/63 [============================>.] - ETA: 0s - loss: 1.3327 - accuracy: 0.5618
Case_2 Val_loss did not 